In [1]:
import geopandas as gpd
import pickle

# Prepare Allen Coral Atlas File
Using the benthic map file downloaded from allen coral atlas, this notebook prepares a pickle file to be used for pulling satellite data. The benthic map for a given region can be downloaded here: https://allencoralatlas.org/atlas

In [2]:
allen_coral_df = gpd.read_file("C:/Users/Danielle/Desktop/nasa-coral-health-observation/data/Benthic-Map/benthic.gpkg")
allen_coral_df

,class,geometry
0,Coral/Algae,"POLYGON ((-86.80897 20.97881, -86.80892 20.978..."
1,Rubble,"POLYGON ((-86.80739 20.97845, -86.80735 20.978..."
2,Rubble,"POLYGON ((-86.80874 20.97912, -86.80874 20.979..."
3,Rubble,"POLYGON ((-86.81004 20.97831, -86.80996 20.978..."
4,Rubble,"POLYGON ((-86.81094 20.97791, -86.81085 20.977..."
...,...,...
859099,Seagrass,"POLYGON ((-97.25076 21.50746, -97.25071 21.507..."
859100,Seagrass,"POLYGON ((-97.25206 21.50728, -97.25193 21.507..."
859101,Rubble,"POLYGON ((-97.25211 21.50688, -97.25197 21.506..."
859102,Coral/Algae,"POLYGON ((-97.24955 21.51227, -97.24955 21.512..."


Create a centroid column from the bounding box provided in the geometry column.

In [3]:
allen_coral_df['centroid_column'] = allen_coral_df.centroid

C:\Users\Danielle\AppData\Local\Temp\ipykernel_44624\2445519379.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  allen_coral_df['centroid_column'] = allen_coral_df.centroid


Extract longitude and latitude from each centroid.

In [5]:
allen_coral_df['long'] = allen_coral_df['centroid_column'].x
allen_coral_df['lat'] = allen_coral_df['centroid_column'].y
allen_coral_df

,class,geometry,centroid_column,long,lat
0,Coral/Algae,"POLYGON ((-86.80897 20.97881, -86.80892 20.978...",POINT (-86.80888 20.97867),-86.808880,20.978671
1,Rubble,"POLYGON ((-86.80739 20.97845, -86.80735 20.978...",POINT (-86.80737 20.97842),-86.807372,20.978424
2,Rubble,"POLYGON ((-86.80874 20.97912, -86.80874 20.979...",POINT (-86.80884 20.97868),-86.808843,20.978677
3,Rubble,"POLYGON ((-86.81004 20.97831, -86.80996 20.978...",POINT (-86.80993 20.97817),-86.809931,20.978174
4,Rubble,"POLYGON ((-86.81094 20.97791, -86.81085 20.977...",POINT (-86.81091 20.97784),-86.810908,20.977840
...,...,...,...,...,...
859099,Seagrass,"POLYGON ((-97.25076 21.50746, -97.25071 21.507...",POINT (-97.25074 21.50744),-97.250737,21.507442
859100,Seagrass,"POLYGON ((-97.25206 21.50728, -97.25193 21.507...",POINT (-97.25185 21.50704),-97.251850,21.507038
859101,Rubble,"POLYGON ((-97.25211 21.50688, -97.25197 21.506...",POINT (-97.25198 21.50674),-97.251982,21.506739
859102,Coral/Algae,"POLYGON ((-97.24955 21.51227, -97.24955 21.512...",POINT (-97.2515 21.50915),-97.251505,21.509150


Function to create a subsample of data points from the allen coral atlas. The coral/algae and non-coral classes will be equally represented.

In [6]:
def getAllenCoralSample(allen_df, numSamples):
    # Rename all Non-Coral classes
    allen_df.loc[allen_df["class"] == "Rubble", "class"] = 'Non-Coral'
    allen_df.loc[allen_df["class"] == "Rock", "class"] = 'Non-Coral'
    allen_df.loc[allen_df["class"] == "Sand", "class"] = 'Non-Coral'
    allen_df.loc[allen_df["class"] == "Microalgal Mats", "class"] = 'Non-Coral'
    allen_df.loc[allen_df["class"] == "Seagrass", "class"] = 'Non-Coral'
    
    # Sample Coral/Algae and Non-Coral classes equally
    allen_sample = allen_df.groupby('class').apply(lambda x: x.sample(numSamples))
    
    return allen_sample

In [7]:
allen_sample = getAllenCoralSample(allen_coral_df, 25000)

C:\Users\Danielle\AppData\Local\Temp\ipykernel_44624\3860211343.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  allen_sample = allen_df.groupby('class').apply(lambda x: x.sample(numSamples))


In [9]:
allen_sample['class'].value_counts()

class
Coral/Algae    25000
Non-Coral      25000
Name: count, dtype: int64

In [10]:
allen_sample.to_pickle('C:/Users/Danielle/Desktop/nasa-coral-health-observation/data/mesoamerica_subset.pkl')